<h1>Previsão de receita por pessoa com base no Censo</h1>

Neste notebook contém um projeto desenvolvido durante o curso "Machine Learning e Data Science com Python de A a Z" da instituição Udemy.

OBJETIVO DA ANÁLISE:
- Comparar algoritmos de Machine Learning que possam prever se a receita ultrapassa 50 mil/ano com base nos dados do censo. 
  Também conhecido como conjunto de dados "Census Income".
  - > 50K, <= 50K.
  
Informações do conjunto de dados:
- Data set pode ser encontrado no link: http://archive.ics.uci.edu/ml/datasets/Adult
- A extração foi feita por Barry Becker do banco de dados do Censo de 1994.
- A tarefa de previsão é determinar se uma pessoa ganha mais de 50K por ano.

Algoritmos de Classificação utilizados:
- Naive Bayes
- Árvore de Decisão
- Random Forest
- KNN
- Regressão Logística
- SVM
- Redes Neurais Artificiais


In [1]:
import warnings
warnings.filterwarnings('ignore')

<h1>Importando a base de dados</h1>

In [2]:
import pandas as pd

In [3]:
# importando a base de dados
# para uma melhor compreensão, será alterado o nome das colunas
base = pd.read_csv('census.csv', names = ['idade', 'tipo_emprego', 'caracteristica', 'educacao', 'anos_estudo', \
                                        'estado_civil', 'ocupacao', 'parentesco', 'raça', 'sexo', 'ganho_capital', \
                                        'perda_capital','hrs_trabalhada_semana', 'pais_origem', 'renda_anual'], header = 0)

In [4]:
base.head(10)

,idade,tipo_emprego,caracteristica,educacao,anos_estudo,estado_civil,ocupacao,parentesco,raça,sexo,ganho_capital,perda_capital,hrs_trabalhada_semana,pais_origem,renda_anual
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


<h1>Explorando os dados</h1>

In [5]:
# verificando o tipo de dados de cada coluna
base.dtypes

idade                     int64
tipo_emprego             object
caracteristica            int64
educacao                 object
anos_estudo               int64
estado_civil             object
ocupacao                 object
parentesco               object
raça                     object
sexo                     object
ganho_capital             int64
perda_capital             int64
hrs_trabalhada_semana     int64
pais_origem              object
renda_anual              object
dtype: object

In [6]:
# quantidade de linhas e colunas do dataFrame
base.shape
print("O dataset tem {} linhas e {} colunas".format(base.shape[0], base.shape[1]))

O dataset tem 32561 linhas e 15 colunas


In [7]:
# utilizando a função Info para obter um resumo sobre o dataset 
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   idade                  32561 non-null  int64 
 1   tipo_emprego           32561 non-null  object
 2   caracteristica         32561 non-null  int64 
 3   educacao               32561 non-null  object
 4   anos_estudo            32561 non-null  int64 
 5   estado_civil           32561 non-null  object
 6   ocupacao               32561 non-null  object
 7   parentesco             32561 non-null  object
 8   raça                   32561 non-null  object
 9   sexo                   32561 non-null  object
 10  ganho_capital          32561 non-null  int64 
 11  perda_capital          32561 non-null  int64 
 12  hrs_trabalhada_semana  32561 non-null  int64 
 13  pais_origem            32561 non-null  object
 14  renda_anual            32561 non-null  object
dtypes: int64(6), object

<h1>Atributos previsores e classe</h1>

In [8]:
# criando uma variável para armazenar os atributos previsores
previsores = base.iloc[:, 0:14].values

In [9]:
previsores

array([[39, ' State-gov', 77516, ..., 0, 40, ' United-States'],
       [50, ' Self-emp-not-inc', 83311, ..., 0, 13, ' United-States'],
       [38, ' Private', 215646, ..., 0, 40, ' United-States'],
       ...,
       [58, ' Private', 151910, ..., 0, 40, ' United-States'],
       [22, ' Private', 201490, ..., 0, 20, ' United-States'],
       [52, ' Self-emp-inc', 287927, ..., 0, 40, ' United-States']],
      dtype=object)

In [10]:
# criando uma variável para armazenar o atributo classe
classe = base.iloc[:, 14].values

In [11]:
classe

array([' <=50K', ' <=50K', ' <=50K', ..., ' <=50K', ' <=50K', ' >50K'],
      dtype=object)

<h1>Transformação de variáveis categóricas</h1>

In [12]:
# LabelEncoder para transformar as variáveis categóricas em numéricas
from sklearn.preprocessing import LabelEncoder

In [13]:
# transformando as variáveis categóricas dos atributos previsores
labelencoder_previsores = LabelEncoder()

previsores[:, 1] = labelencoder_previsores.fit_transform(previsores[:, 1])
previsores[:, 3] = labelencoder_previsores.fit_transform(previsores[:, 3])
previsores[:, 5] = labelencoder_previsores.fit_transform(previsores[:, 5])
previsores[:, 6] = labelencoder_previsores.fit_transform(previsores[:, 6])
previsores[:, 7] = labelencoder_previsores.fit_transform(previsores[:, 7])
previsores[:, 8] = labelencoder_previsores.fit_transform(previsores[:, 8])
previsores[:, 9] = labelencoder_previsores.fit_transform(previsores[:, 9])
previsores[:, 13] = labelencoder_previsores.fit_transform(previsores[:, 13])

In [14]:
previsores

array([[39, 7, 77516, ..., 0, 40, 39],
       [50, 6, 83311, ..., 0, 13, 39],
       [38, 4, 215646, ..., 0, 40, 39],
       ...,
       [58, 4, 151910, ..., 0, 40, 39],
       [22, 4, 201490, ..., 0, 20, 39],
       [52, 5, 287927, ..., 0, 40, 39]], dtype=object)

In [15]:
# transformando a variável categórica do atributo classe
labelencoder_classe = LabelEncoder()
classe = labelencoder_classe.fit_transform(classe)

In [16]:
# <= 50K tem o valor = 0
# > 50K tem o valor = 1
classe

array([0, 0, 0, ..., 0, 0, 1])

<h1>Escalonamento dos atributos</h1>

In [17]:
# Aplicar uma fórmula para os atributos na mesma escala
# Utilizando a Padronização x= x - média(x) / desvioPadrão(x)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

In [18]:
previsores

array([[ 0.03067056,  2.15057856, -1.06361075, ..., -0.21665953,
        -0.03542945,  0.29156857],
       [ 0.83710898,  1.46373585, -1.008707  , ..., -0.21665953,
        -2.22215312,  0.29156857],
       [-0.04264203,  0.09005041,  0.2450785 , ..., -0.21665953,
        -0.03542945,  0.29156857],
       ...,
       [ 1.42360965,  0.09005041, -0.35877741, ..., -0.21665953,
        -0.03542945,  0.29156857],
       [-1.21564337,  0.09005041,  0.11095988, ..., -0.21665953,
        -1.65522476,  0.29156857],
       [ 0.98373415,  0.77689313,  0.92989258, ..., -0.21665953,
        -0.03542945,  0.29156857]])

<h1>Spliting</h1>

In [19]:
# Dividindo as variáveis em treino e teste
# 85% dados para treino e 15% dados para teste
from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, \
                                                                                             test_size=0.15, random_state=0)

In [20]:
# Imprimindo os resultados
print("{0:0.2f}% nos dados de treino".format((len(previsores_treinamento)/len(base.index)) * 100))
print("{0:0.2f}% nos dados de teste".format((len(previsores_teste)/len(base.index)) * 100))

85.00% nos dados de treino
15.00% nos dados de teste


In [76]:
len(previsores_treinamento)

27676

In [78]:
len(classe_treinamento)

27676

In [77]:
len(previsores_teste)

4885

In [79]:
len(classe_teste)

4885

<h1>Naive Bayes</h1>

O algoritmo “Naive Bayes” é um classificador probabilístico.
Ele recebe o nome de “naive” (ingênuo) porque desconsidera a correlação entre as variáveis (features).

Classificação de textos, filtragem de SPAM e análise de sentimento em redes sociais são algumas das muitas aplicações para esse algoritmo.
O algoritmo é muito robusto para previsões em tempo real, pois precisa de poucos dados para realizar a classificação. Se existe a necessidade de correlacionar fatores, esse algoritmo tende a falhar nas predições.

### Construindo e treinando o modelo

In [120]:
# Importando o algoritmo
from sklearn.naive_bayes import GaussianNB

In [121]:
# Criando o modelo preditivo
classificador = GaussianNB()

In [122]:
# Treinando o modelo
classificador.fit(previsores_treinamento, classe_treinamento)

GaussianNB(priors=None, var_smoothing=1e-09)

In [123]:
# testando o modelo
previsoes = classificador.predict(previsores_teste)

### Verificando a exatidão no modelo

In [124]:
# precisão do modelo
from sklearn.metrics import confusion_matrix, accuracy_score
precisao = accuracy_score(classe_teste, previsoes)
precisao 

0.8057318321392016

In [125]:
# matriz de confusão
# diagonal principal (quantidade de acertos)
matriz = confusion_matrix(classe_teste, previsoes)
matriz

array([[3528,  165],
       [ 784,  408]], dtype=int64)

<h1>Árvore de Decisão</h1>

Uma árvore de decisão utiliza base de dados histórica e geralmente começa com um único nó, que se divide em possíveis resultados. Cada um desses resultados leva a nós adicionais, que se ramificam em outras possibilidades. Assim, cria-se uma forma de árvore. Esses algoritmos são considerados um dos melhores e mais utilizados métodos de aprendizagem supervisionada, pois nos dão modelos preditivos de alta precisão, estabilidade e facilidade de interpretação. Ao contrário dos modelos lineares, eles mapeiam muito bem relações não-lineares.

### Construindo e treinando o modelo

In [126]:
# Importando o algoritmo
from sklearn.tree import DecisionTreeClassifier

In [127]:
# Criando o modelo preditivo
classificador = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)

In [128]:
# Treinando o modelo
classificador.fit(previsores_treinamento, classe_treinamento)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [129]:
# testando o modelo
previsoes = classificador.predict(previsores_teste)

### Verificando a exatidão no modelo

In [130]:
# precisão do modelo
precisao = accuracy_score(classe_teste, previsoes)
precisao

0.8128966223132037

In [132]:
# matriz de confusão
# diagonal principal (quantidade de acertos)
matriz = confusion_matrix(classe_teste, previsoes)
matriz

array([[3239,  454],
       [ 460,  732]], dtype=int64)

<h1>Random Forest</h1>

Esse algoritmo irá criar muitas árvores de decisão, de maneira aleatória, formando o que podemos enxergar como uma floresta, onde cada árvore será utilizada na escolha do resultado final. É um método de aprendizagem de máquina versátil e capaz de executar tarefas de regressão e de classificação. Ele também aplica métodos de redução dimensional, trata valores faltantes, valores anómalos (‘outliers’) e outras etapas essenciais da exploração de dados.
É um tipo de método de aprendizado de ‘ensemble’, onde um grupo de modelos fracos são combinados para formar um modelo mais forte.

### Construindo e treinando o modelo

In [133]:
# Importando o algoritmo
from sklearn.ensemble import RandomForestClassifier

In [134]:
# Criando o modelo preditivo
classificador = RandomForestClassifier(n_estimators = 40, criterion = 'entropy', random_state = 0)

In [135]:
# Treinando o modelo
classificador.fit(previsores_treinamento, classe_treinamento)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [136]:
# testando o modelo
previsoes = classificador.predict(previsores_teste)

### Verificando a exatidão no modelo

In [137]:
# precisão do modelo
precisao = accuracy_score(classe_teste, previsoes)
precisao

0.8483111566018424

In [138]:
# matriz de confusão
# diagonal principal (quantidade de acertos)
matriz = confusion_matrix(classe_teste, previsoes)
matriz

array([[3419,  274],
       [ 467,  725]], dtype=int64)

<h1>KNN</h1>

O KNN (K-Nearest Neighbor) tem o objetivo de determinar a qual grupo uma determinada amostra vai pertencer com base nas amostras vizinhas. Os exemplos de treinamento são armazenados e a previsão é feita somente quando um novo registro precisa ser classificado. Diferente dos outros algoritmos, ele não constrói um modelo, apenas faz o cálculo da distância.

### Construindo e treinando o modelo

In [139]:
# Importando o algoritmo
from sklearn.neighbors import KNeighborsClassifier

In [140]:
classificador = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)

In [141]:
classificador.fit(previsores_treinamento, classe_treinamento)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [142]:
# testando
previsoes = classificador.predict(previsores_teste)

### Verificando a exatidão 

In [143]:
# precisão 
precisao = accuracy_score(classe_teste, previsoes)
precisao

0.8219037871033776

In [144]:
# matriz de confusão
# diagonal principal (quantidade de acertos)
matriz = confusion_matrix(classe_teste, previsoes)
matriz

array([[3331,  362],
       [ 508,  684]], dtype=int64)

<h1>Regressão Logística</h1>

Esse algoritmo mede a relação entre a variável dependente categórica e uma ou mais variáveis independentes, estimando as probabilidades usando uma função logística. Analisa diferentes aspectos ou variáveis de um objeto para depois determinar uma classe na qual ele se encaixa melhor.

### Construindo e treinando o modelo

In [145]:
# Importando o algoritmo
from sklearn.linear_model import LogisticRegression

In [146]:
# Criando o modelo preditivo
classificador = LogisticRegression(solver='lbfgs')

In [147]:
# Treinando o modelo
classificador.fit(previsores_treinamento, classe_treinamento)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [148]:
# testando o modelo
previsoes = classificador.predict(previsores_teste)

### Verificando a exatidão no modelo

In [149]:
# precisão do modelo
precisao = accuracy_score(classe_teste, previsoes)
precisao

0.8184237461617195

In [150]:
# matriz de confusão
# diagonal principal (quantidade de acertos)
matriz = confusion_matrix(classe_teste, previsoes)
matriz

array([[3464,  229],
       [ 658,  534]], dtype=int64)

<h1>SVM - Máquinas de Vetores de Suporte</h1>

Uma máquina de vetores de suporte (SVM) desenvolve o modelo tomando as entradas de treinamento, mapeando elas no espaço multidimensional e utilizando regressão para encontrar um hiperplano (um hiperplano é uma superfície em espaço de n dimensões que o separa em duas metades de espaço) que melhor separa duas classes de entradas. Uma vez que esse modelo tenha sido treinada, ele é capaz de avaliar novas entradas em relação ao hiperplano divisor e classificá-las em uma entre duas categorias.

### Construindo e treinando o modelo

In [151]:
# Importando o algoritmo
from sklearn.svm import SVC

In [152]:
# Criando o modelo preditivo
classificador = SVC(kernel = 'linear', random_state = 1)

In [153]:
# Treinando o modelo
classificador.fit(previsores_treinamento, classe_treinamento)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.001,
    verbose=False)

In [154]:
# testando o modelo
previsoes = classificador.predict(previsores_teste)

### Verificando a exatidão no modelo

In [155]:
# precisão do modelo
precisao = accuracy_score(classe_teste, previsoes)
precisao

0.813510747185261

In [156]:
# matriz de confusão
# diagonal principal (quantidade de acertos)
matriz = confusion_matrix(classe_teste, previsoes)
matriz

array([[3598,   95],
       [ 816,  376]], dtype=int64)

<h1>Redes Neurais Artificiais</h1>

Redes Neurais Artificiais são técnicas computacionais que apresentam um modelo matemático inspirado na estrutura neural de organismos inteligentes e que adquirem conhecimento através da experiência. As redes neurais são compostas por várias unidades de processamento. Mesmo sendo usadas para resolverem problemas complexos, será utilizado como um exemplo.

## Redes Neurais com sklearn

### Construindo e treinando o modelo

In [157]:
# Importando o algoritmo
from sklearn.neural_network import MLPClassifier

In [158]:
# Criando o modelo preditivo
classificador = MLPClassifier(verbose=True, max_iter=1000, tol= 0.000010)

In [159]:
# Treinando o modelo
classificador.fit(previsores_treinamento, classe_treinamento)

Iteration 1, loss = 0.42422199
Iteration 2, loss = 0.35391352
Iteration 3, loss = 0.33337361
Iteration 4, loss = 0.32615847
Iteration 5, loss = 0.32334609
Iteration 6, loss = 0.32199615
Iteration 7, loss = 0.32069130
Iteration 8, loss = 0.31987058
Iteration 9, loss = 0.31921785
Iteration 10, loss = 0.31855495
Iteration 11, loss = 0.31829660
Iteration 12, loss = 0.31744305
Iteration 13, loss = 0.31682105
Iteration 14, loss = 0.31642574
Iteration 15, loss = 0.31595524
Iteration 16, loss = 0.31551810
Iteration 17, loss = 0.31505792
Iteration 18, loss = 0.31467126
Iteration 19, loss = 0.31452719
Iteration 20, loss = 0.31366123
Iteration 21, loss = 0.31346877
Iteration 22, loss = 0.31309825
Iteration 23, loss = 0.31256578
Iteration 24, loss = 0.31218147
Iteration 25, loss = 0.31199080
Iteration 26, loss = 0.31146256
Iteration 27, loss = 0.31125270
Iteration 28, loss = 0.31060299
Iteration 29, loss = 0.31008520
Iteration 30, loss = 0.31017551
Iteration 31, loss = 0.30921991
Iteration 32, los

Iteration 253, loss = 0.28027126
Iteration 254, loss = 0.28015698
Iteration 255, loss = 0.28053309
Iteration 256, loss = 0.28002856
Iteration 257, loss = 0.28033172
Iteration 258, loss = 0.28006230
Iteration 259, loss = 0.28097743
Iteration 260, loss = 0.27964005
Iteration 261, loss = 0.27995994
Iteration 262, loss = 0.27953421
Iteration 263, loss = 0.27978835
Iteration 264, loss = 0.27919320
Iteration 265, loss = 0.27938522
Iteration 266, loss = 0.27973609
Iteration 267, loss = 0.27920087
Iteration 268, loss = 0.27978675
Iteration 269, loss = 0.28000207
Iteration 270, loss = 0.27908397
Iteration 271, loss = 0.27875546
Iteration 272, loss = 0.27908424
Iteration 273, loss = 0.27909790
Iteration 274, loss = 0.27937269
Iteration 275, loss = 0.27912809
Iteration 276, loss = 0.27883464
Iteration 277, loss = 0.27875407
Iteration 278, loss = 0.27858891
Iteration 279, loss = 0.27843493
Iteration 280, loss = 0.27896489
Iteration 281, loss = 0.27805010
Iteration 282, loss = 0.27863987
Iteration 

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=1000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=1e-05, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [160]:
# testando o modelo
previsoes = classificador.predict(previsores_teste)

### Verificando a exatidão no modelo

In [161]:
# precisão do modelo
precisao = accuracy_score(classe_teste, previsoes)
precisao

0.8417604912998976

In [163]:
# matriz de confusão
# diagonal principal (quantidade de acertos)
matriz = confusion_matrix(classe_teste, previsoes)
matriz

array([[3414,  279],
       [ 494,  698]], dtype=int64)

## Redes Neurais com Keras

### Construindo e treinando o modelo

In [164]:
from keras.models import Sequential
from keras.layers import Dense

In [165]:
# Criando o modelo preditivo
classificador = Sequential()

In [166]:
# Criação das camadas ocultas e de saída
classificador.add(Dense(units = 8, activation = 'relu', input_dim = 14))
classificador.add(Dense(units = 8, activation = 'relu'))
classificador.add(Dense(units = 1, activation = 'sigmoid'))

In [167]:
# Compilando a rede neural
classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [168]:
# Treinando o modelo
classificador.fit(previsores_treinamento, classe_treinamento, batch_size = 10, epochs = 100)

Epoch 1/100
27676/27676 [==============================] - 2s 69us/step - loss: 0.3941 - accuracy: 0.8228
Epoch 2/100
27676/27676 [==============================] - 2s 66us/step - loss: 0.3355 - accuracy: 0.8413
Epoch 3/100
27676/27676 [==============================] - 2s 66us/step - loss: 0.3288 - accuracy: 0.8453
Epoch 4/100
27676/27676 [==============================] - 2s 66us/step - loss: 0.3260 - accuracy: 0.8465
Epoch 5/100
27676/27676 [==============================] - 2s 66us/step - loss: 0.3241 - accuracy: 0.8467
Epoch 6/100
27676/27676 [==============================] - 2s 66us/step - loss: 0.3224 - accuracy: 0.8495
Epoch 7/100
27676/27676 [==============================] - 2s 68us/step - loss: 0.3218 - accuracy: 0.8497
Epoch 8/100
27676/27676 [==============================] - 2s 66us/step - loss: 0.3209 - accuracy: 0.8503
Epoch 9/100
27676/27676 [==============================] - 2s 67us/step - loss: 0.3202 - accuracy: 0.8504
Epoch 10/100
27676/27676 [====================

In [169]:
# testando o modelo
previsoes = classificador.predict(previsores_teste)
previsoes = (previsoes > 0.5)

### Verificando a exatidão no modelo

In [170]:
# precisão do modelo
precisao = accuracy_score(classe_teste, previsoes)
precisao

0.847697031729785

In [171]:
# matriz de confusão
# diagonal principal (quantidade de acertos)
matriz = confusion_matrix(classe_teste, previsoes)
matriz

array([[3406,  287],
       [ 457,  735]], dtype=int64)

<h1>Conclusão</h1>

Realizando apenas um teste podemos observar que os Algoritmos de Redes Neurais Artificiais e Random Forest tiveram um maior percentual de acerto nessa base de dados. Isso não significa que esses algoritmos sejam os melhores a serem utilizados para prever a renda anual por pessoa dessa base de dados. Para uma melhor compreensão desse resultado, deve ser feito mais testes a fim de comprovar a eficácia desses algoritmos.